# Steps for this analyis: 
1. For each of the TRs for a subject, load in the {beta, t, p} maps 
    - If there is no data for a TR, insert an array of zeros to place hold the data
2. stack the 3d arrays into a 4d array
3. save the 4d array to the correct location in the directory

In [52]:
# imports 
from pathlib import Path
import nibabel as nib
import numpy as np

# Data folder
DATA_FOLDER = Path('/home/fs0/chx061/scratch/data/derivatives/')
SUBJECT     = 'sub-01'
EVS_TYPE    = 'instruction_period_sliding_window'
TR          = '0'
RDM         = 'replay'

# Select the TR directory
TR_DIR = DATA_FOLDER / SUBJECT / 'analysis' / EVS_TYPE

# Get the list of folders
folders = [f for f in TR_DIR.iterdir() if 'TR' in f.name and f.is_dir()]
folders.sort()
folders

[PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR0'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR1'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR2'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR3'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR4'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR5'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR6'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR7'),
 PosixPath('/home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR8'),
 

In [53]:

# /home/fs0/chx061/scratch/data/derivatives/sub-01/analysis/instruction_period_sliding_window/TR0/replay/results-standard-space
# For each folder, get the beta maps from standard space
MAP_TYPE = 't'

video = {}
for folder in folders:
    # print(folder)
    results_std = folder / RDM / 'results-standard-space' / F'{MAP_TYPE}_values_std.nii.gz'
    # print(results_std)
    if not results_std.exists():
        # print('No results-std-space found, adding None to dictionary')
        video[folder] = None
        continue
    else:
        # print('Found results-std-space, adding to dictionary')
        map = nib.load(results_std)
        video[folder] = map.get_fdata()

array_shape = video[folders[0]].shape

# Replace the None with the previous value
for path_idx, path in enumerate(video.keys()):
    if video[path] is None:
        print(f'Found None at index {path_idx}, replacing with zeros')
        video[path] = np.zeros(array_shape)

Found None at index 5, replacing with zeros
Found None at index 7, replacing with zeros


In [54]:
# Stack the 3D arrays
stack = np.stack(list(video.values()), axis=3)

# Create the correct directiory to save the stack
stack_dir = DATA_FOLDER / SUBJECT / 'analysis'  / EVS_TYPE / 'video_stack' / RDM
stack_dir.mkdir(parents=True, exist_ok=True)
stack_dir = DATA_FOLDER / SUBJECT / 'analysis'  / EVS_TYPE / 'video_stack' / RDM / F'{MAP_TYPE}_values_std.nii.gz'

# Save the stack
stack_img = nib.Nifti1Image(stack, map.affine)
nib.save(stack_img, stack_dir)
